In [ ]:
# !pip install imutils
# !pip install super_gradients==3.0.7
# !pip install albumentations 
# !pip install split-folders[full]

In [ ]:
import math
import random
from typing import Dict, List,Tuple
import requests

import numpy as np
import matplotlib.pyplot as plt
import glob
from pathlib import Path, PurePath
import pathlib
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms

from PIL import Image

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split

from imutils import paths

import splitfolders
import textwrap

import super_gradients
from super_gradients.common.object_names import Models
from super_gradients.training import Trainer
from super_gradients.training import training_hyperparams
from super_gradients.training.metrics.classification_metrics import Accuracy, Top5
from super_gradients.training.utils.early_stopping import EarlyStop
from super_gradients.training import models
from super_gradients.training.utils.callbacks import Phase

# Number of classes

Simple utility to fetch the number of classes.

In [ ]:
from pathlib import Path

def count_subdirectories(path: str) -> int:
    """
    Counts the number of subdirectories in the given directory path.
    """
    dir_path = Path(path)
    subdirectories = [f for f in dir_path.iterdir() if f.is_dir()]
    return len(subdirectories)

# Example usage
parent_dir = "../input/100-bird-species/train"
num_subdirectories = count_subdirectories(parent_dir)
print(f"Number of subdirectories in {parent_dir}: {num_subdirectories}")

# Config 

This holds variables for the notebook.

You will define the model, training params, image type, number of classes, and 
relevant directories in this class.

In [ ]:
class config:
    # specify the paths to datasets
    ROOT_DIR = Path('../input/100-bird-species')
    TRAIN_DIR = ROOT_DIR.joinpath('train')
    TEST_DIR = ROOT_DIR.joinpath('test')
    VAL_DIR = ROOT_DIR.joinpath('valid')

    # set the input height and width
    INPUT_HEIGHT = 224
    INPUT_WIDTH = 224

    # set the input heig/ht and width
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    
    IMAGE_TYPE = '.jpg'
    BATCH_SIZE = 128
    MODEL_NAME = 'resnet50'
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    TRAINING_PARAMS = 'training_hyperparams/imagenet_resnet50_train_params'
    
    NUM_CLASSES = num_subdirectories
#     IMAGE_TYPE = 
    
    CHECKPOINT_DIR = 'checkpoints'


# Plot random images

In [ ]:
train_image_path_list = list(sorted(paths.list_images(config.TRAIN_DIR)))
train_image_path_sample = random.sample(population=train_image_path_list, k=20)

def examine_images(images:list):
    num_images = len(images)
    num_rows = int(math.ceil(num_images/5))
    num_cols = 5
    
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(30, 30),tight_layout=True)
    axs = axs.ravel()

    for i, image_path in enumerate(images[:num_images]):
        image = Image.open(image_path)
        label = PurePath(image_path).parent.name
        axs[i].imshow(image)
        axs[i].set_title(f"Bird: {label}", fontsize=25)
        axs[i].axis('off')
    plt.show()

examine_images(train_image_path_sample)

In [ ]:
!head ../input/100-bird-species/birds.csv

# Augmentations


In [ ]:
# initialize our data augmentation functions
resize = transforms.Resize(size=(config.INPUT_HEIGHT,config.INPUT_WIDTH))
make_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=config.IMAGENET_MEAN, std=config.IMAGENET_STD)
center_cropper = transforms.CenterCrop((config.INPUT_HEIGHT,config.INPUT_WIDTH))
random_horizontal_flip = transforms.RandomHorizontalFlip(p=0.5)
random_vertical_flip = transforms.RandomVerticalFlip(p=0.5)
random_rotation = transforms.RandomRotation(degrees=180)
random_crop = transforms.RandomCrop(size=(170,170))
random_erasing = transforms.RandomErasing()
augmix = transforms.AugMix(severity = 4, mixture_width=4, alpha=0.65)

# randomly_choose_one 

# initialize our training and validation set data augmentation pipeline
train_transforms = transforms.Compose([
  resize, 
  augmix,
  center_cropper,
  random_crop,
  random_horizontal_flip,
  random_vertical_flip,
  random_rotation,
  make_tensor,
  normalize
])

val_transforms = transforms.Compose([resize, make_tensor, normalize])

# Show what one image looks like after augmentation

In [ ]:
def apply_transform(img: Image, transform) -> np.ndarray:
    """
    Applies a transform to a PIL Image and returns a numpy array of the transformed image.

    Args:
        img (PIL.Image): The input image to transform.
        transform (torchvision.transforms.Compose): The transform to apply to the image.

    Returns:
        np.ndarray: A numpy array representing the transformed image.
    """
    # Apply the transform to the image
    if isinstance(transform, torchvision.transforms.Compose):
        # Apply PyTorch transform to image array
        transformed_image = train_transforms(img)

    elif isinstance(transform, A.Compose):
        # Apply Albumentations transform to image array
        img_array = np.array(img)
        transformed_image = transform(image=img_array)["image"]

    # Convert the image tensor to a numpy array and transpose the axes to (height, width, channels)
    img_array = transformed_image.numpy().transpose((1, 2, 0))

    # Clip the pixel values to the range [0, 1]
    img_array = np.clip(img_array, 0, 1)

    return img_array


def visualize_transform(image: np.ndarray, original_image: np.ndarray = None) -> None:
    """
    Visualize the transformed image.

    Args:
        image (np.ndarray): A NumPy array representing the transformed image.
        original_image (np.ndarray, optional): A NumPy array representing the original image. Defaults to None.
    """
    fontsize = 18
    
    if original_image is None:
        # Create a plot with 1 row and 2 columns.
        f, ax = plt.subplots(1, 2, figsize=(12, 12))

        # Show the transformed image in the first column.
        ax[0].imshow(image)
    else:
        # Create a plot with 1 row and 2 columns.
        f, ax = plt.subplots(1, 2, figsize=(12, 12))

        # Show the original image in the first column.
        ax[0].imshow(original_image)
        ax[0].set_title('Original image', fontsize=fontsize)
        
        # Show the transformed image in the second column.
        ax[1].imshow(image)
        ax[1].set_title('Transformed image', fontsize=fontsize)
        
img = Image.open(train_image_path_list[0])
img_array = apply_transform(img, train_transforms)

visualize_transform(img_array, original_image=img)

# Datasets and Dataloadrer

In [ ]:
def create_dataloaders(train_dir: str, val_dir: str, test_dir: str,
                       train_transform: transforms.Compose, val_transform: transforms.Compose,
                       test_transform: transforms.Compose, batch_size: int, num_workers: int = 2):
    """Creates training and validation DataLoaders.

    Args:
        train_dir (str): Path to training data.
        val_dir (str): Path to validation data.
        test_dir (str): Path to test data.
        train_transform (torchvision.transforms.Compose): Transformation pipeline for training data.
        val_transform (torchvision.transforms.Compose): Transformation pipeline for validation data.
        test_transform (torchvision.transforms.Compose): Transformation pipeline for test data.
        batch_size (int): Number of samples per batch in each of the DataLoaders.
        num_workers (int): Number of workers per DataLoader.

    Returns:
        Tuple of (train_dataloader, val_dataloader, test_dataloader, class_names).
    """
    # Use ImageFolder to create dataset
    train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)
    test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

    print(f"[INFO] Training dataset contains {len(train_dataset)} samples.")
    print(f"[INFO] Validation dataset contains {len(val_dataset)} samples.")
    print(f"[INFO] Test dataset contains {len(test_dataset)} samples.")

    # Get class names
    class_names = train_dataset.classes
    print(f"[INFO] Dataset contains {len(class_names)} labels.")
    
    print("[INFO] creating training and validation set dataloaders...")
    train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      drop_last=True,
      num_workers=num_workers,
      pin_memory=True,
      persistent_workers=True)


In [ ]:
def create_dataloaders(
    train_dir: str, 
    val_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    val_transform:  transforms.Compose,
    test_transform:  transforms.Compose,
    batch_size: int, 
    num_workers: int=2
):
  """Creates training and validation DataLoaders.
  Args:
    train_dir: Path to training data.
    val_dir: Path to validation data.
    transform: Transformation pipeline.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.
  Returns:
    A tuple of (train_dataloader, val_dataloader, class_names).
  """
  # Use ImageFolder to create dataset
  train_data = datasets.ImageFolder(train_dir, transform=train_transform)
  val_data = datasets.ImageFolder(val_dir, transform=val_transform)
  test_data = datasets.ImageFolder(test_dir, transform=val_transform)  

  print(f"[INFO] training dataset contains {len(train_data)} samples...")
  print(f"[INFO] validation dataset contains {len(val_data)} samples...")
  print(f"[INFO] test dataset contains {len(test_data)} samples...")

  # Get class names
  class_names = train_data.classes
  print(f"[INFO] dataset contains {len(class_names)} labels...")

  # Turn images into data loaders
  print("[INFO] creating training and validation set dataloaders...")
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      drop_last=True,
      num_workers=num_workers,
      pin_memory=True,
      persistent_workers=True
  )
  val_dataloader = DataLoader(
      val_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
      drop_last=False,
      persistent_workers=True
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
      drop_last=False,
      persistent_workers=True
  )

  return train_dataloader, val_dataloader, test_dataloader, class_names

In [ ]:
train_dataloader, valid_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=config.TRAIN_DIR,
                                                                     val_dir=config.VAL_DIR,
                                                                     test_dir=config.TEST_DIR,
                                                                     train_transform=train_transforms,
                                                                     val_transform=val_transforms,
                                                                     test_transform=val_transforms,
                                                                     batch_size=config.BATCH_SIZE)

NUM_CLASSES = len(class_names)

# Training Params

In [ ]:
training_params =  training_hyperparams.get(config.TRAINING_PARAMS)

In [ ]:
training_params

In [ ]:
# To reduce clutter in the notebook I've turned the verbosity off, you can turn it on to see the full output
early_stop_acc = EarlyStop(Phase.VALIDATION_EPOCH_END, monitor="Accuracy", mode="max", patience=7, verbose=False)
early_stop_val_loss = EarlyStop(Phase.VALIDATION_EPOCH_END, monitor="LabelSmoothingCrossEntropyLoss", mode="min", patience=7, verbose=False)

training_params["train_metrics_list"] = [Accuracy(), Top5()]
training_params["valid_metrics_list"] = [Accuracy(), Top5()]
training_params["phase_callbacks"] = [early_stop_acc, early_stop_val_loss]

# Set the silent mode to True to reduce clutter in the notebook, you can turn it on to see the full output
training_params["silent_mode"] = True
# We'll turn off the use of exponential moving average and zero weight decay on bias and batch norm
# training_params['ema'] = False
training_params['zero_weight_decay_on_bias_and_bn'] = False
training_params["optimizer"] = 'Adam'

training_params["criterion_params"] = {'smooth_eps': 0.20}
training_params["max_epochs"] = 250
training_params["initial_lr"] = 0.0001

# Get model

In [ ]:
model = models.get(config.MODEL_NAME, num_classes = config.NUM_CLASSES, pretrained_weights='imagenet')

# Instantiate trainer

In [ ]:
full_model_trainer = Trainer(experiment_name='0_Baseline_Experiment', ckpt_root_dir=config.CHECKPOINT_DIR)

# Train model

In [ ]:
full_model_trainer.train(model=model, 
              training_params=training_params, 
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)

In [ ]:
import os
best_full_model = models.get(config.MODEL_NAME,
                        num_classes=config.NUM_CLASSES,
                        checkpoint_path=os.path.join(full_model_trainer.checkpoints_dir_path, "average_model.pth"))

# Evaluate on test set

In [ ]:
full_model_trainer.test(model=best_full_model,
            test_loader=test_dataloader,
            test_metrics_list=['Accuracy', 'Top5'])

# Looks like we're able to achieve 99.16% accuracy on the test set!

# Plot predictions

In [ ]:
def pred_and_plot_image(image_path: str, 
                        subplot: Tuple[int, int, int],  # subplot tuple for `subplot()` function
                        class_names: List[str] = class_names,
                        model: torch.nn.Module = best_full_model,
                        image_size: Tuple[int, int] = (config.INPUT_HEIGHT, config.INPUT_WIDTH),
                        transform: torchvision.transforms = None,
                        device: torch.device=config.DEVICE):

    if isinstance(image_path, pathlib.PosixPath):
        img = Image.open(image_path)
    else: 
        img = Image.open(requests.get(image_path, stream=True).raw)

    # create transformation for image (if one doesn't exist)
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=config.IMAGENET_MEAN,
                                 std=config.IMAGENET_STD),
        ])
    transformed_image = transform(img)

    # make sure the model is on the target device
    model.to(device)

    # turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = transformed_image.unsqueeze(dim=0)

        # make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(device))

    # convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # actual label
    ground_truth = PurePath(image_path).parent.name

    # plot image with predicted label and probability 
    plt.subplot(*subplot)
    plt.imshow(img)
    if isinstance(image_path, pathlib.PosixPath):
        title = f"Ground Truth: {ground_truth} | Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
    else:
        title = f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
    plt.title("\n".join(textwrap.wrap(title, width=20)))  # wrap text using textwrap.wrap() function
    plt.axis(False)
    

def plot_random_test_images(model):
    num_images_to_plot = 30
    test_image_path_list = list(Path(config.TEST_DIR).glob("*/*.jpg")) # get list all image paths from test data 
    test_image_path_sample = random.sample(population=test_image_path_list, # go through all of the test image paths
                                           k=num_images_to_plot) # randomly select 'k' image paths to pred and plot

    # set up subplots
    num_rows = int(np.ceil(num_images_to_plot / 5))
    fig, ax = plt.subplots(num_rows, 5, figsize=(15, num_rows * 3))
    ax = ax.flatten()

    # Make predictions on and plot the images
    for i, image_path in enumerate(test_image_path_sample):
        pred_and_plot_image(model=model, 
                            image_path=image_path,
                            class_names=class_names,
                            subplot=(num_rows, 5, i+1),  # subplot tuple for `subplot()` function
                            image_size=(config.INPUT_HEIGHT, config.INPUT_WIDTH))

    # adjust spacing between subplots
    plt.subplots_adjust(wspace=1)
    plt.show()

# Predict on images from internet

In [ ]:
plot_random_test_images(best_full_model)

In [ ]:
#Baltimore oriole
image_url = 'https://www.allaboutbirds.org/guide/assets/og/75258971-1200px.jpg'
pred_and_plot_image(image_path= image_url, subplot=(1, 1, 1))

In [ ]:
#Sri Lankan blue magpie
image_url = 'https://cdn.download.ams.birds.cornell.edu/api/v1/asset/252002651/1800'
pred_and_plot_image(image_path= image_url, subplot=(1, 1, 1))

In [ ]:
#BLACK THROATED BUSHTIT
image_url = 'http://aabird.weebly.com/uploads/1/2/0/6/12063687/339097369.jpg'
pred_and_plot_image(image_path= image_url, subplot=(1, 1, 1))

# Your homework

Copy/fork this notebook and try some different architectures.

If you have a question you can leave a comment on this notebook, or visit the community and post it in the [Q&A section](https://www.deeplearningdaily.community/c/qanda/8).

## Use a different pretrained model

You can change the model you use. Take a look at the [SG model zoo](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/training/Computer_Vision_Models_Pretrained_Checkpoints.md)

For example, if you wanted to use RegNet you would do the following:

```
resnet_imagenet_model = models.get(model_name='regnetY800', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_regnetY_train_params')
```

Note you can also pass 'model_name=regnetY200', 'model_name=regnetY400', 'model_name=regnetY600' to try a variety of the architecture

For ResNet50, you would do:

```
resnet_imagenet_model = models.get(model_name='resnet50', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_resnet50_train_params')
```

Note you can also pass 'model_name=resnet18' or 'model_name=resnet34' to try a variety of the architecture

For MobileNetV2, you would do:

```
mobilenet_imagenet_model = models.get(model_name='mobilenet_v2', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_mobilenetv2_train_params')
```

For MobileNetV3, you would do:

```
mobilenet_imagenet_model = models.get(model_name='mobilenet_v3_large', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_mobilenetv3_train_params')
```

Note you can also pass 'model_name=mobilenet_v3_small' to try a variety of the architecture


For ViT, you would do:


```
vit_imagenet_model = models.get(model_name='vit_base', num_classes=NUM_CLASSES, pretrained_weights='imagenet')
vit_params =  training_hyperparams.get("training_hyperparams/imagenet_vit_train_params")
```

Note you can also pass 'model_name=vit_large' to try a variety of the architecture


I encourage you play around with different optimizers, all you have to do is change the value of `training_params["optimizer"]`. You can use one of ['Adam','SGD','RMSProp'] out of the box. You can play around with the optimizer params as well.

In general, play and tweak around the training recipies...

## Training recipes

SuperGradients has a number of [training recipes](https://github.com/Deci-AI/super-gradients/tree/master/src/super_gradients/recipes) you can use. [See here](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/recipes/training_hyperparams/default_train_params.yaml) for more information about the training params.

If you're using Weights and Biases to track your experiments, you would do the following

```
sg_logger: wandb_sg_logger
sg_logger_params:
project_name: <YOUR PROJECT NAME>
entity: algo
api_server: https://wandb.research.deci.ai
save_checkpoints_remote: True
save_tensorboard_remote: True
save_logs_remote: True
```